# Data Description

For Aiming Folder:

| File Type | Granularity | Description | Role in the Project |
| --- | --- | --- | --- |
| AimingData.csv | Per Trial | Contains subject demographics (Gender, DOB) and trial conditions (visCond, surface, distance). This is the main metadata and conditions file | The Base/Core DataFrame. We will build everything onto this |
| s...timeInfoData.csv | Per Trial | Summary of movement timing for each trial (onset, offset, movTime) | Mostly redundant, as this info is in other files, but can be used for verification |
| s...grasp_paramData.csv | Per Trial | Calculated Features that summarises the entire grasp for a trail (eg. MaxGrip Aperture MGA, Max Velocity MVel). The signal column is 'grasp'. | These are precalculated summaries we can use directly |
| s...reach_paramData.csv | Per Marker, Per Trial | Calculated Feature for individual markers (index, thumb, etc.) during the reach. Here we have multiple rows per trial | We need to reshape to fit the one row per trial format |
| s...trajData.csv | Per Time-Step, Per Trial | The raw 3D coordinates over time for all markers. | Too granular for a standard ML model directly. We use this for advance feature engineering or deep learning (eg. transformers). The paramData filess are the summary of this raw data|

# First we will merge all this information into a single DataFrame

1. Combine all the single subject files: First we will read all the files for individual subjects (s001, etc) and concatenate them into a single DataFrame for each data type.
2. Handle the Multi Row Per Trial Data (reach_paramData): This feature is too long. We will pivot it to a wide format so theres only one row per trial. For each parameter in that file (MVel1, Macc, etc), we will create new columns based on the signal (eg. MVel_index, MVel_thumb, MAcc_index, MAcc_thumb).
3. Merge everything together

In [4]:
import pandas as pd
import glob
import os

data_path = "C:/CourseWork/Dissertation/Data/Aiming/filtered_data/"


## Step 1: Load and combine the Parameter Files

In [ ]:
# 1a. Load and combine all GRASP parameter files
grasp_param_files = glob.glob(os.path.join(data_path, "s*grasp_paramData.csv"))
grasp_df_list = [pd.read_csv(f) for f in grasp_param_files]
grasp_params_full = pd.concat(grasp_df_list, ignore_index=True)

print(f"Combined {len(grasp_param_files)} grasp_param files. Shape: {grasp_params_full.shape}")

# Drop 'signal' column as it's constant ('grasp') and redundant
grasp_params_full = grasp_params_full.drop(columns=["signal"])


"""
Find all CSV files in the specified folder that match the pattern "s*grasp_paramData.csv".

Read each file and store the data in a list.

Combine all the individual data tables into a single large table (row-wise).

Display how many files were combined and the shape (rows × columns) of the final table.

Remove the column named "signal" because it has the same value ("grasp") in every row and is not useful.
"""

Combined 18 grasp_param files. Shape: (2423, 50)


In [ ]:
# 1b. Load and combine all REACH parameter files
reach_param_files = glob.glob(os.path.join(data_path, "s*reach_paramData.csv"))
reach_df_list = [pd.read_csv(f) for f in reach_param_files]
reach_params_long = pd.concat(reach_df_list, ignore_index=True)

print(f"Combined {len(reach_param_files)} reach_param files. Shape: {reach_params_long.shape}")

"""
Search for all CSV files in the folder that match "s*reach_paramData.csv".

Read each file and store its contents in a list.

Concatenate all the tables from the list into one long table (row-wise).

Display how many files were combined and the size (rows × columns) of the final table.
"""

Combined 18 reach_param files. Shape: (12115, 39)


## Step 2: Reshape the Reach Parameter Data (Pivot)

In [ ]:
# Identify columns that are constant for a trail (the identifiers)
id_vars = ['subjName', 'trialN', 'visCond', 'surface', 'distance', 'onset', 'offset', 'movTime']

# Indentify columns that contain the actual values we want to pivot
value_vars = [col for col in reach_params_long.columns if col not in id_vars and col != "signal"]

# Pivot the table
reach_params_wide = reach_params_long.pivot_table(
    index=["subjName", "trialN"],
    columns="signal",
    values=value_vars
)

# The pivot creates a multi level column index. We will flatten it.
# This will rename columns from ("MVel", "index") to "MVel_index"
reach_params_wide.columns = [f"{val}_{sig}" for val, sig in reach_params_wide.columns]
reach_params_wide.reset_index(inplace=True)     # Make the subjName and trialN columns again

print(f"Pivoted reach_pparams data. Shape: {reach_params_wide.shape}")

"""
Define identifying columns (id_vars) that stay the same for each trial (e.g., subject name, trial number, condition info, timing).

Identify measurement columns (value_vars) by excluding the identifier columns and the "signal" column.

Pivot the data so that each signal type becomes a separate column:

Rows are grouped by subject and trial number.

Each value column is spread across the different signal types.

Flatten the column headers by combining the value name and signal name (e.g., "MVel_index").

Restore subject and trial columns as regular columns (not index).

Print the shape (rows x columns) of the reshaped data.
"""

Pivoted reach_pparams data. Shape: (2423, 152)


## Step 3: Load the Main Metadata and Merge Everything

In [ ]:
# 3a. Load the main AimingData file
df_master = pd.read_csv(os.path.join(data_path, "AimingData.csv"))
print(f"Loaded master data. Shape: {df_master.shape}")

# Define the keys for merging
merge_keys = ["subjName", "trialN"]

# 3b. Merge with the full grasp parameters.
'''We will fo an inner merge to ensure we only keep trials present in all datasets. We also need to handle redundant columns. 'visCond', 'surface', etc. are in both. We will drop them from the parameter files before merging'''
cols_to_drop_from_grasp = ['visCond', 'surface', 'distance', 'onset', 'offset', 'movTime']
df_master = pd.merge(
    df_master,
    grasp_params_full.drop(columns=cols_to_drop_from_grasp, errors="ignore"),
    on=merge_keys,
    how="inner"
)
print(f"Shape after merging grasp params: {df_master.shape}")

"""
Read the main dataset (AimingData.csv) into memory.

Print the shape of this master dataset.

Define merge keys: the columns "subjName" and "trialN" will be used to join datasets.

Drop redundant columns (e.g., "visCond", "surface", etc.) from the grasp parameter data to avoid duplication.

Merge the master dataset with the cleaned grasp parameter data:

Use an inner join so only trials present in both datasets are kept.

Join based on the "subjName" and "trialN" columns.

Print the new shape of the merged dataset.
"""

Loaded master data. Shape: (2423, 22)
Shape after merging grasp params: (2423, 63)


In [ ]:
# 3c. Merge with the wide reach parameters
df_master = pd.merge(
    df_master,
    reach_params_wide,
    on=merge_keys,
    how="inner"
)
print(f"Final shape after merging reach params: {df_master.shape}")

"""
Merge the current master dataset with the wide-format reach parameters:

Use an inner join to keep only common trials.

Join on "subjName" and "trialN".

Print the final shape of the dataset after merging.
"""

Final shape after merging reach params: (2423, 213)


## Step 4: Final Cleanup and Save

In [10]:
# Check for issues
print("\nFirst 5 rows of the final dataframe:")
df_master


First 5 rows of the final dataframe:


,subjName,trialN,visCond,surface,distance,DOB,Gender,Dominant.Eye,clear.logMAR,mono.logMAR,...,timeToYmax_index,timeToYmax_knuck,timeToYmax_target,timeToYmax_thumb,timeToYmax_wrist,timeToZmax_index,timeToZmax_knuck,timeToZmax_target,timeToZmax_thumb,timeToZmax_wrist
0,1,1,clear,black,three,28/03/2003,female,left,-0.06,0.02,...,2.041667,2.058333,2.041667,2.041667,2.075000,2.758333,2.816667,2.583333,2.608333,2.600000
1,1,2,clear,black,two,28/03/2003,female,left,-0.06,0.02,...,2.175000,2.200000,2.175000,2.183333,2.225000,2.883333,3.216667,2.858333,2.550000,2.500000
2,1,3,clear,wood,three,28/03/2003,female,left,-0.06,0.02,...,1.675000,1.683333,1.675000,1.675000,1.708333,2.600000,2.625000,2.500000,2.466667,2.108333
3,1,4,clear,wood,one,28/03/2003,female,left,-0.06,0.02,...,1.700000,1.733333,1.700000,1.700000,1.775000,2.300000,1.941667,1.933333,1.941667,1.941667
4,1,5,clear,wood,two,28/03/2003,female,left,-0.06,0.02,...,2.058333,2.083333,2.066667,2.075000,2.116667,2.775000,2.841667,2.625000,2.641667,2.633333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,18,140,mono,black,three,17/03/05,female,left,-0.04,0.02,...,2.166667,2.158333,2.150000,2.133333,2.166667,3.691667,3.666667,2.691667,2.708333,2.716667
2419,18,141,mono,black,two,17/03/05,female,left,-0.04,0.02,...,2.225000,2.166667,2.200000,1.908333,2.141667,3.658333,3.650000,2.433333,2.425000,2.441667
2420,18,142,mono,black,one,17/03/05,female,left,-0.04,0.02,...,2.450000,2.391667,2.400000,2.341667,2.391667,2.666667,3.350000,2.666667,2.666667,2.666667
2421,18,143,mono,wood,one,17/03/05,female,left,-0.04,0.02,...,2.375000,2.375000,2.366667,2.350000,2.383333,2.675000,3.191667,2.675000,2.691667,2.700000


In [ ]:
print(f"Final combined dataframe has {df_master.shape[0]} rows and {df_master.shape[1]} columns.")
print("Each row represents a single trail.")

# Save the combined dataframe to a single csv
df_master.to_csv("aiming_master_dataset.csv", index=False)
print("\nSuccessfully saved the combined data to 'aiming_master_dataset.csv'")

Final combined dataframe has 2423 rows and 213 columns.
Each row represents a single trail.

Successfully saved the combined data to 'aiming_master_dataset.csv'


In [12]:
print(df_master.columns)

Index(['subjName', 'trialN', 'visCond', 'surface', 'distance', 'DOB', 'Gender',
       'Dominant.Eye', 'clear.logMAR', 'mono.logMAR',
       ...
       'timeToYmax_index', 'timeToYmax_knuck', 'timeToYmax_target',
       'timeToYmax_thumb', 'timeToYmax_wrist', 'timeToZmax_index',
       'timeToZmax_knuck', 'timeToZmax_target', 'timeToZmax_thumb',
       'timeToZmax_wrist'],
      dtype='object', length=213)


In [13]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)

df = pd.read_csv("aiming_master_dataset.csv")
df.head(3)

,subjName,trialN,visCond,surface,distance,DOB,Gender,Dominant.Eye,clear.logMAR,mono.logMAR,bino.logMAR,clear.logCS,mono.logCS,bino.logCS,clear.disparity,mono.disparity,bino.disparity,MGA_x,MVel_x,movTime,timeMDecToOffset_x,timeMVelToOffset,FX,FY,FZ,FXVel,FYVel,FZVel,FVel,FAcc,MVel_y,MAcc,MDec,timeMVel,timeMAcc,timeMDec,pathLength,Xmax,Ymax,Zmax,timeToXmax,timeToYmax,timeToZmax,XlocMinN,YlocMinN,ZlocMinN,XlocMaxN,YlocMaxN,ZlocMaxN,timeMAccToMVel,timeMVelToMDec,timeMDecToOffset_y,FGA,MGA_y,timeMGA,timeMVelToMGA,timeMGAToMDec,timeMGAToOffset,MGAVel,MGAAcc,FGOf,FGOt,FGOs,FAcc_index,FAcc_knuck,FAcc_target,FAcc_thumb,FAcc_wrist,FVel_index,FVel_knuck,FVel_target,FVel_thumb,FVel_wrist,FX_index,FX_knuck,FX_target,FX_thumb,FX_wrist,FXVel_index,FXVel_knuck,FXVel_target,FXVel_thumb,FXVel_wrist,FY_index,FY_knuck,FY_target,FY_thumb,FY_wrist,FYVel_index,FYVel_knuck,FYVel_target,FYVel_thumb,FYVel_wrist,FZ_index,FZ_knuck,FZ_target,FZ_thumb,FZ_wrist,FZVel_index,FZVel_knuck,FZVel_target,FZVel_thumb,FZVel_wrist,MAcc_index,MAcc_knuck,MAcc_target,MAcc_thumb,MAcc_wrist,MDec_index,MDec_knuck,MDec_target,MDec_thumb,MDec_wrist,MVel_index,MVel_knuck,MVel_target,MVel_thumb,MVel_wrist,XlocMaxN_index,XlocMaxN_knuck,XlocMaxN_target,XlocMaxN_thumb,XlocMaxN_wrist,XlocMinN_index,XlocMinN_knuck,XlocMinN_target,XlocMinN_thumb,XlocMinN_wrist,Xmax_index,Xmax_knuck,Xmax_target,Xmax_thumb,Xmax_wrist,YlocMaxN_index,YlocMaxN_knuck,YlocMaxN_target,YlocMaxN_thumb,YlocMaxN_wrist,YlocMinN_index,YlocMinN_knuck,YlocMinN_target,YlocMinN_thumb,YlocMinN_wrist,Ymax_index,Ymax_knuck,Ymax_target,Ymax_thumb,Ymax_wrist,ZlocMaxN_index,ZlocMaxN_knuck,ZlocMaxN_target,ZlocMaxN_thumb,ZlocMaxN_wrist,ZlocMinN_index,ZlocMinN_knuck,ZlocMinN_target,ZlocMinN_thumb,ZlocMinN_wrist,Zmax_index,Zmax_knuck,Zmax_target,Zmax_thumb,Zmax_wrist,pathLength_index,pathLength_knuck,pathLength_target,pathLength_thumb,pathLength_wrist,timeMAcc_index,timeMAcc_knuck,timeMAcc_target,timeMAcc_thumb,timeMAcc_wrist,timeMAccToMVel_index,timeMAccToMVel_knuck,timeMAccToMVel_target,timeMAccToMVel_thumb,timeMAccToMVel_wrist,timeMDec_index,timeMDec_knuck,timeMDec_target,timeMDec_thumb,timeMDec_wrist,timeMDecToOffset_index,timeMDecToOffset_knuck,timeMDecToOffset_target,timeMDecToOffset_thumb,timeMDecToOffset_wrist,timeMVel_index,timeMVel_knuck,timeMVel_target,timeMVel_thumb,timeMVel_wrist,timeMVelToMDec_index,timeMVelToMDec_knuck,timeMVelToMDec_target,timeMVelToMDec_thumb,timeMVelToMDec_wrist,timeToXmax_index,timeToXmax_knuck,timeToXmax_target,timeToXmax_thumb,timeToXmax_wrist,timeToYmax_index,timeToYmax_knuck,timeToYmax_target,timeToYmax_thumb,timeToYmax_wrist,timeToZmax_index,timeToZmax_knuck,timeToZmax_target,timeToZmax_thumb,timeToZmax_wrist
0,1,1,clear,black,three,28/03/2003,female,left,-0.06,0.02,0.26,1.76,1.8,1.56,60,240,240,0.130996,1.329988,3.375,1.058333,1.316667,0.006617,0.035757,0.379251,-0.002046,0.003522,0.048539,0.048710,2.559395,1.329988,8.564791,-5.751450,2.058333,1.850000,2.316667,0.570574,0.030561,0.208276,0.401785,0.000000,2.041667,2.591667,6,7,5,6,6,5,0.208333,0.258334,1.058333,0.124676,0.130996,2.841667,0.783334,-0.525000,0.533333,0.231187,2.309931,0.083703,0.627714,0.115113,2.415172,1.973688,2.390412,2.651702,1.309014,0.059358,0.048710,0.058162,0.039402,0.037346,0.056956,0.030372,0.005520,-0.043722,0.009914,0.005370,0.002804,0.000660,-0.009462,-0.006474,0.031533,0.043944,0.111087,0.039980,0.066543,0.003092,0.005074,0.015803,0.003953,0.011003,0.415778,0.341818,0.394489,0.342725,0.236571,0.059034,0.048364,0.055970,0.038045,0.035096,9.405101,7.012234,8.395760,7.863698,7.874024,-7.135599,-5.860425,-5.483523,-6.084781,-6.016711,1.334769,1.352856,1.320023,1.329909,1.302514,7.0,9.0,13.0,6.0,8.0,8.0,10.0,14.0,6.0,7.0,0.065630,0.067940,0.036547,-0.003599,0.034435,8.0,5.0,8.0,7.0,6.0,9.0,6.0,9.0,8.0,7.0,0.210317,0.213981,0.221762,0.206236,0.159403,7.0,6.0,10.0,7.0,9.0,8.0,6.0,11.0,7.0,9.0,0.428769,0.346045,0.398716,0.378211,0.260092,0.618421,0.582191,0.495232,0.564074,0.520648,1.850000,1.858333,1.850000,1.866

In [1]:
# df.info